In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential, Model
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding,Dropout, Bidirectional, Input, merge, Flatten, Reshape
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import SGD, Adam, RMSprop
from keras import backend
import keras
import json
from data_utils import *
from model import *

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
SENTENCE_LENGTH = 20
VOCAB_SIZE = 30000

def load_cornell_movie_data(start_line, limit, vocab_dict):
    sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, start_line, limit)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_twitter_data(start_line, limit, vocab_dict):
    sentences = read_sentences("ShortenTwitterAsciiCorpus.txt")[start_line:start_line+limit]
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_friends_data(start_line, limit, vocab_dict):
    sentences = read_lines("friends-final.txt", "	", 2, 5, start_line = 0, limit = 10000)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec
def load_fake_data(start_line, limit, vocab_dict):
    sentences = [["hi"], ["how","is", "it", "going" ], ["i","am", "good"], ["that","is", "great", "to","hear"],["hello"],["how","is", "it", "going" ], ["hey"], ["hey", "there"]  ]
    for i in range(4):
        sentences += sentences
    #sentences = [ ["i'm", "not", "bad"], ["that's", "great", "to","hear"]]
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

In [3]:
#https://github.com/oswaldoludwig/Seq2seq-Chatbot-for-Keras/blob/master/train_bot.py


In [4]:
vocab_to_idx, idx_to_vocab, embeding_matrix = get_vocab('glove.6B.300d.txt', VOCAB_SIZE)

Loaded GloVe!


In [5]:
print(embeding_matrix.shape)

(30000, 300)


In [6]:
#vocab_dict, idx_dict = load_vocab("vocab_dict.json")
#model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 100, 3)
model = create_model_with_attention(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, embeding_matrix, 300, 100, 3)
#print(model.summary())
model.load_weights("0new_model.h5")

opt = RMSprop(lr=0.0001,rho=0.9, epsilon=None, decay=1e-7)
model.compile(loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
#model.summary()

C:\Users\Gavin\Projects\ML\MLPractice\chatbot\model.py:37: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  output_att = merge([inputs, att_prob ], mode='mul')
C:\ProgramData\Anaconda3\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [ ]:

print(backend.get_value(opt.lr))

1e-04


In [ ]:
batch_size = 4000
start_point = 5000
#21000
round = 0

#x_vec, y_vec = load_friends_data(start_point, batch_size, vocab_to_idx) 
while True: 
    print("Getting data:", start_point, batch_size) 
    x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_to_idx)

    if len(x_vec) == 0:
        start_point = 0
        x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_to_idx)
        round += 1 
    y = np.argmax(y_vec, axis=2)
    print(" ".join([idx_to_vocab[x] for x in y[0] ]) )
    last_acc = 0
    i = 0
    while True:
        history = model.fit(x_vec, y_vec, batch_size=50, nb_epoch=20, shuffle = True)
        model.save_weights(str(i%2)+"new_model.h5")
        
        if abs( history.history["acc"][0] - last_acc ) < 0.02:
            if backend.get_value(opt.lr) == 0.0001:
                backend.set_value(opt.lr, 0.00001)
            else:
                backend.set_value(opt.lr, 0.0001)
        else:
            backend.set_value(opt.lr, 0.001)
            
        if history.history["acc"][0] > 0.95:
            backend.set_value(opt.lr, 0.00001)
        print("set learning rate: " , backend.get_value(opt.lr) )
        #if  abs( history.history["acc"][0] - last_acc ) >= 0.009 and history.history["acc"][0] < 0.985:
        if history.history["acc"][0] < 0.99:
            print(last_acc, history.history["acc"][0], " continue for the same dataset until requirement is satisfied.")
        else:
            break
        last_acc = history.history["acc"][0]
        
        i += 1
        print("--------------------------------------------------------------------------")
        print("Round:", round,"Data Start Point", start_point)
        print("--------------------------------------------------------------------------")
    start_point += batch_size

Getting data: 5000 4000
(1763, 20, 30000)
                                do you see ?


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/20
1763/1763 [==============================] - 24s 14ms/step - loss: 7.1544 - acc: 0.3414
Epoch 2/20
1763/1763 [==============================] - 22s 12ms/step - loss: 6.4967 - acc: 0.3605
Epoch 3/20
1763/1763 [==============================] - 20s 11ms/step - loss: 6.0713 - acc: 0.3732
Epoch 4/20
1763/1763 [==============================] - 21s 12ms/step - loss: 5.7272 - acc: 0.3847
Epoch 5/20
1763/1763 [==============================] - 23s 13ms/step - loss: 5.4491 - acc: 0.3941
Epoch 6/20
1763/1763 [==============================] - 22s 13ms/step - loss: 5.2120 - acc: 0.4068
Epoch 7/20
1763/1763 [==============================] - 23s 13ms/step - loss: 5.0095 - acc: 0.4189
Epoch 8/20
1763/1763 [==============================] - 23s 13ms/step - loss: 4.8364 - acc: 0.4265
Epoch 9/20
1763/1763 [==============================] - 20s 11ms/step - loss: 4.6816 - acc: 0.4381
Epoch 10/20
 150/1763 [=>............................] - ETA: 20s - loss: 4.3131 - acc: 0.4517

In [ ]:
#do you clean
sen = "are you stupid ?"
sen =  sen .lower().split(" ")
print(sen)
vec = sentence_to_vec([sen], vocab_to_idx, 'UNK', SENTENCE_LENGTH)

res = model.predict(vec)
#print(res)
vec_y = np.argmax(res, axis=2)
" ".join([idx_to_vocab[x] for x in vec_y[0] ])
#if idx_dict[x]!= 'UNK'

In [ ]:
vocab_to_idx["hey"]